In [1]:
import findspark
findspark.init('/home/gerardo-rodriguez/spark-4.0.0-bin-hadoop3')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('rec').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/21 09:59:00 WARN Utils: Your hostname, Lanz-Lenovo, resolves to a loopback address: 127.0.1.1; using 192.168.1.145 instead (on interface wlp2s0)
25/08/21 09:59:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/21 09:59:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [5]:
data = spark.read.csv('movielens_ratings.csv', header=True, inferSchema=True)

In [6]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows


In [7]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [9]:
train_data, test_data = data.randomSplit([0.8,0.2])

In [11]:
als = ALS(maxIter=5, regParam=0.01, userCol='userId', 
          itemCol='movieId', ratingCol='rating')

In [12]:
model = als.fit(train_data)

In [13]:
predictions = model.transform(test_data)

In [15]:
predictions.show()

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|      0|   3.0|    28| 0.6077937|
|      3|   1.0|    28|-2.6962116|
|      0|   1.0|    27| 1.1632158|
|      2|   1.0|    26| 2.7993083|
|      2|   1.0|    12| 0.7231444|
|      6|   1.0|    12| 0.5090272|
|      0|   1.0|     6| 1.7105253|
|      1|   1.0|     6|-1.5873984|
|      2|   3.0|     6| 0.9445233|
|      6|   1.0|    20| 1.4757555|
|      4|   1.0|    19| 0.3064979|
|      0|   1.0|    15|0.20442574|
|      2|   3.0|     9| 0.9484575|
|      6|   1.0|     9| 2.2680888|
|      4|   2.0|     8| 2.3008533|
|      4|   1.0|    23|0.22504213|
|      1|   1.0|     7|  3.308064|
|      6|   2.0|    11| 2.7129114|
|      3|   3.0|    14|-1.1251056|
|      6|   1.0|    14|0.33361858|
+-------+------+------+----------+
only showing top 20 rows


In [16]:
eva = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [17]:
rmse = eva.evaluate(predictions)

In [18]:
print('RMSE')
rmse

RMSE


2.228775070889769

In [20]:
single_user = test_data.filter(test_data['userId'] == 1).select(['movieId', 'userId'])

In [21]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     37|     1|
|     44|     1|
|     67|     1|
|     72|     1|
|     76|     1|
|     86|     1|
|     97|     1|
+-------+------+



In [22]:
recommendations = model.transform(single_user)

In [24]:
recommendations.orderBy('prediction', ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     37|     1|  5.3925805|
|     72|     1|  3.7380908|
|     86|     1|  2.0462391|
|     97|     1|   0.565827|
|     76|     1|  0.4197312|
|     44|     1|-0.06715532|
|     67|     1| -0.5690796|
+-------+------+-----------+

